# Limestone IDW Validation Notebook

This notebook reproduces the validation for:
- Baseline isotropic IDW (k, p tuning).
- Enhanced: first-order trend plane + anisotropic IDW of residuals (k, p, theta, ratio tuning).

Validation schemes:
- Standard LOOCV (baseline only, reference).
- Spatially buffered LOOCV over multiple buffer fractions.
- Spatial block CV (3x3 grid) with nested tuning on training folds.

Relies on functions in `analyze_limestone.py` and utilities in `validate_limestone_models.py`.

In [17]:
import os
from importlib import reload

import analyze_limestone as al
import validate_limestone_models as val

# Working paths
csv_file = "input/limestone_layers_phase_1_input.csv"
# here = os.getcwd()
# data_path = os.path.join(here, csv_file)
print('Using data:', csv_file)

# Reload to pick up any recent edits
# reload(al); reload(val)


Using data: input/limestone_layers_phase_1_input.csv


In [18]:
# Load rows and derive measured points for rock head and thickness
rows = val.read_rows(csv_file)
rock_points, thick_points = val.derive_points(rows)
print(f'Rock points: {len(rock_points)} | Thick points: {len(thick_points)}')


Rock points: 27 | Thick points: 27


In [19]:
# Helper to display tables nicely in the notebook without pandas
from IPython.display import display, HTML

def display_table(title, rows, cols):
    """
    title: str
    rows: List[Dict[str, Any]]-like (dicts)
    cols: list of (header, key)
    """
    html = f'<h3>{title}</h3>'
    html += "<table border='1' cellpadding='4' cellspacing='0'>"
    html += '<tr>' + ''.join(f'<th>{hdr}</th>' for hdr, _ in cols) + '</tr>'
    for r in rows:
        html += '<tr>'
        for _, key in cols:
            v = r.get(key, '')
            if isinstance(v, float):
                s = f"{v:.3f}"
            else:
                s = str(v)
            html += f'<td>{s}</td>'
        html += '</tr>'
    html += '</table>'
    display(HTML(html))


In [20]:
# Run validations
k_grid = [3, 4, 5, 6, 8, 10, 12]
p_grid = [1.0, 2.0, 3.0, 4.0]

# Standard LOOCV (baseline only)
if len(rock_points) >= 3:
    k_r, p_r, m_r = al.loocv_idw(rock_points, k_grid, p_grid)
else:
    k_r, p_r, m_r = None, None, {'MAE': float('nan'), 'RMSE': float('nan'), 'R2': float('nan')}

if len(thick_points) >= 3:
    k_t, p_t, m_t = al.loocv_idw(thick_points, k_grid, p_grid)
else:
    k_t, p_t, m_t = None, None, {'MAE': float('nan'), 'RMSE': float('nan'), 'R2': float('nan')}

loocv_rows = [
    {'variable': 'rock_head', 'method': 'baseline', 'scheme': 'loocv', 'k': k_r, 'p': p_r, 'theta': '', 'ratio': '', 'buffer': '',
     'MAE': m_r['MAE'], 'RMSE': m_r['RMSE'], 'R2': m_r['R2']},
    {'variable': 'thickness', 'method': 'baseline', 'scheme': 'loocv', 'k': k_t, 'p': p_t, 'theta': '', 'ratio': '', 'buffer': '',
     'MAE': m_t['MAE'], 'RMSE': m_t['RMSE'], 'R2': m_t['R2']},
]

# Buffered LOOCV sensitivity (both methods)
rock_buf_base = val.buffered_cv_sweep(rock_points, method='baseline')
rock_buf_enh  = val.buffered_cv_sweep(rock_points, method='enhanced')
thick_buf_base = val.buffered_cv_sweep(thick_points, method='baseline')
thick_buf_enh  = val.buffered_cv_sweep(thick_points, method='enhanced')

# Spatial block CV (3x3 grid)
rock_block_base = val.block_cv(rock_points, method='baseline', nx=3, ny=3)
rock_block_enh  = val.block_cv(rock_points, method='enhanced', nx=3, ny=3)
thick_block_base = val.block_cv(thick_points, method='baseline', nx=3, ny=3)
thick_block_enh  = val.block_cv(thick_points, method='enhanced', nx=3, ny=3)

# Aggregate rows for CSV and displays
all_rows = []
def add_rows(variable, method, items):
    for it in items:
        all_rows.append({
            'variable': variable, 'method': method, 'scheme': it.get('scheme',''),
            'k': it.get('k',''), 'p': it.get('p',''), 'theta': it.get('theta',''), 'ratio': it.get('ratio',''),
            'buffer': it.get('buffer',''), 'grid': it.get('grid',''),
            'MAE': it['MAE'], 'RMSE': it['RMSE'], 'R2': it['R2'],
            'folds': it.get('folds',''), 'n_test_points': it.get('n_test_points',''),
        })

add_rows('rock_head', 'baseline', loocv_rows[0:1])
add_rows('thickness', 'baseline', loocv_rows[1:2])
add_rows('rock_head', 'baseline', rock_buf_base)
add_rows('rock_head', 'enhanced', rock_buf_enh)
add_rows('thickness', 'baseline', thick_buf_base)
add_rows('thickness', 'enhanced', thick_buf_enh)
add_rows('rock_head', 'baseline', [{'scheme': 'block_cv_3x3', **rock_block_base, 'grid': '3x3'}])
add_rows('rock_head', 'enhanced', [{'scheme': 'block_cv_3x3', **rock_block_enh, 'grid': '3x3'}])
add_rows('thickness', 'baseline', [{'scheme': 'block_cv_3x3', **thick_block_base, 'grid': '3x3'}])
add_rows('thickness', 'enhanced', [{'scheme': 'block_cv_3x3', **thick_block_enh, 'grid': '3x3'}])

# Write combined CSV summary
out_path = os.path.join(here, 'limestone_validation_results.csv')
val.write_summary_csv(out_path, all_rows)
print('Wrote:', out_path)


Wrote: /Users/reneboygarcia/Downloads/limestone-idw-analysis/limestone_validation_results.csv


In [21]:
# Display compact tables
rock_rows = [r for r in all_rows if r['variable'] == 'rock_head']
thick_rows = [r for r in all_rows if r['variable'] == 'thickness']

def mk_rows(items):
    out = []
    for r in items:
        out.append({
            'Method': r['method'],
            'Scheme': r['scheme'],
            'Params': f"k={r['k']}, p={r['p']}, th={r['theta']}, ra={r['ratio']}",
            'MAE': r['MAE'], 'RMSE': r['RMSE'], 'R2': r['R2'],
        })
    return out

display_table('Rock head validation (all schemes)', mk_rows(rock_rows),
              [('Method','Method'), ('Scheme','Scheme'), ('Params','Params'), ('MAE','MAE'), ('RMSE','RMSE'), ('R2','R2')])
display_table('Thickness validation (all schemes)', mk_rows(thick_rows),
              [('Method','Method'), ('Scheme','Scheme'), ('Params','Params'), ('MAE','MAE'), ('RMSE','RMSE'), ('R2','R2')])


Method,Scheme,Params,MAE,RMSE,R2
baseline,loocv,"k=6, p=1.0, th=, ra=",5.923,6.905,0.302
baseline,buffered_loocv@10%,"k=8, p=1.0, th=, ra=",6.055,7.216,0.238
baseline,buffered_loocv@20%,"k=6, p=4.0, th=, ra=",5.944,7.220,0.237
baseline,buffered_loocv@30%,"k=12, p=1.0, th=, ra=",8.832,10.771,-0.698
baseline,buffered_loocv@40%,"k=10, p=1.0, th=, ra=",9.061,10.790,-0.705
enhanced,buffered_loocv@10%,"k=3, p=3.0, th=30.0, ra=2.0",5.478,6.885,0.306
enhanced,buffered_loocv@20%,"k=3, p=4.0, th=150.0, ra=3.0",5.980,7.329,0.213
enhanced,buffered_loocv@30%,"k=12, p=1.0, th=30.0, ra=3.0",11.563,13.270,-1.578
enhanced,buffered_loocv@40%,"k=12, p=1.0, th=0.0, ra=3.0",12.695,14.808,-2.210
baseline,block_cv_3x3,"k=9.333333333333334, p=1.3333333333333333, th=, ra=",7.664,9.106,-0.214


Method,Scheme,Params,MAE,RMSE,R2
baseline,loocv,"k=8, p=1.0, th=, ra=",6.738,7.711,-0.013
baseline,buffered_loocv@10%,"k=10, p=2.0, th=, ra=",6.752,7.981,-0.085
baseline,buffered_loocv@20%,"k=12, p=1.0, th=, ra=",7.054,7.756,-0.025
baseline,buffered_loocv@30%,"k=10, p=1.0, th=, ra=",7.435,8.123,-0.124
baseline,buffered_loocv@40%,"k=4, p=1.0, th=, ra=",6.993,8.156,-0.133
enhanced,buffered_loocv@10%,"k=3, p=1.0, th=0.0, ra=1.5",6.435,8.096,-0.116
enhanced,buffered_loocv@20%,"k=3, p=1.0, th=30.0, ra=2.0",6.702,8.382,-0.197
enhanced,buffered_loocv@30%,"k=6, p=1.0, th=0.0, ra=2.0",6.763,8.322,-0.180
enhanced,buffered_loocv@40%,"k=6, p=3.0, th=0.0, ra=1.0",10.332,11.889,-1.408
baseline,block_cv_3x3,"k=9.0, p=1.0, th=, ra=",7.398,8.125,-0.124


## Expert summary (Geotechnical + Data Science)

- **Evidence strength**: Indicative to moderately strong for method ranking; limited by N=27 per variable.
- **Rock head**: Under spatially fair CV (3x3 blocks), the enhanced method (trend + anisotropic residuals) improves MAE/RMSE vs baseline. R² remains near zero/negative, reflecting local variability and sparse coverage.
- **Thickness**: Methods are very similar under fair CV; enhanced shows slight MAE gains but RMSE ~ tie; R² slightly negative.
- **Implications**: Prefer enhanced method for rock head mapping away from borings; either method is acceptable for thickness. Use spatial CV in reporting.
- **Recommendations**: Add geologic covariates to trend, consider variogram-informed kriging for anisotropy, increase observation density in poorly constrained zones, and report prediction intervals.